In [ ]:
df.columns

In [ ]:
df_clean =df.drop_duplicates(subset=['Nom','Prénom'],keep='last')
df_clean.columns = ['Horodateur', 'Nom', 'Prénom', 'Adresse_Mail','Choix1','Raison_Choix1','Choix2','Raison_Choix2',
                   'Choix3','Equipe','OS','Memoire','Github','Last']

In [ ]:
s= {
     df_clean.loc[i,"Nom"]+'_'+df_clean.loc[i,"Prénom"]: [df_clean.loc[i,f'Choix{j}'] for j in range(1,4) ] for i in df_clean.index
}
s

In [ ]:
import random


In [ ]:
def fill_all_student_projects(list_students_projects,
                              list_all_students,
                              method_ranking ):
    student_queue = sorted([y for y in list_all_students if y not in list_students_projects], key=lambda k: method_ranking(k))
    return(list_students_projects+student_queue)
    
def projects_ranking(df, list_mails=[], list_projects=df_clean["Choix1"].unique(),
                     method_class1=len,method_class2= lambda x : random.random()):
   
    df_clean =df.drop_duplicates(subset=['Nom','Prénom'],keep='last')
    df_clean =  df_clean.dropna()
    df_clean.columns = ['Horodateur', 'Nom', 'Prénom', 'Adresse_Mail','Choix1','Raison_Choix1','Choix2','Raison_Choix2',
                   'Choix3','Equipe','OS','Memoire','Github','Last']
    #filtre adresse mail
    if list_mails: 
        print("Filtre activé")
        df_clean[df_clean['Adresse_Mail'].isin(list_mails)]
    #A mettre en paramètre ? avec webscrapping
    list_all_students  = set([ x[0] +"_"+x[1] for x in df_clean.loc[:,["Nom","Prénom"]].values])
    df_clean['score1']=df_clean["Raison_Choix1"].apply(lambda x : method_class1(x))
    df_clean['score2']=df_clean["Raison_Choix2"].apply(lambda x : method_class1(x))
    
    
    vows_projects = {project :[ x[0]+'_'+x[1] 
          for x in df_clean.loc[df_clean["Choix1"]==project,].sort_values(by='score1',ascending=False).loc[:,["Nom","Prénom"]].values]+
                     [ x[0]+'_'+x[1] 
          for x in df_clean.loc[df_clean["Choix2"]==project,].sort_values(by='score2',ascending=False).loc[:,["Nom","Prénom"]].values] 
      
for project in  list_projects}
    return({ project : fill_all_student_projects(vows_projects[project], list_all_students,method_class2) for project in vows_projects})
    
    #on complete les classements de chaque projet pour avoir un classement complet
    
   

In [ ]:
def remove_student_from_waiting_list(applicant,waiting_list):
    """
    Creating a new list of the waiting_list's keys without the key applicant
    """
    return [x for x in waiting_list if x != applicant]


def update_matchings(student, choice,matchings,rankings):
    """
    Update the allocation of the project choice by adding applicant to it and then sort it according to
    the project's vows (contained in rankings)
    """
    matchings[choice].append(student)
    #Selecting the students according to the project's ranking order if they are currently match with this project
    return [x for x in rankings[choice] if x in matchings[choice]]

def stable_matching_algorithm_unbalanced_class(students, rankings, places):
    #initially all the students are waiting to be assign to a project
    waiting_list = [applicants for applicants in students]
    #initially all the projects are empty
    matchings = {choice: [] for choice in places}

    while waiting_list:
        for applicant in waiting_list.copy():
            if not students[applicant]:
                waiting_list = remove_student_from_waiting_list(applicant,waiting_applicants)
                continue
            #each student propose to his first vow
            choice = students[applicant].pop(0)
            #If the project chosen by the student still has some available places :
            if len(matchings[choice]) < places[choice]:
                matchings[choice] = update_matchings(applicant, choice,matchings,rankings)
                waiting_list = remove_student_from_waiting_list(applicant,waiting_list)
            #If not we check if the project prefers the student to its current last member and 
            # update the attributions according to the project's vows
            else:
                if rankings[choice].index(applicant) < rankings[choice].index(matchings[choice][-1]):
                    matchings[choice] = update_matchings(applicant, choice,matchings,rankings)
                    waiting_list = remove_student_from_waiting_list(applicant,waiting_list)
                    waiting_list.append(matchings[choice].pop())

    return matchings

In [ ]:

places ={'3 - OPA': 8,
 '1 - DST Airlines': 6,
 '2 - NY News': 7,
 '4 - Itinéraire de vacances': 5,
 '5 - Satisfaction client': 10}
stable_matching_algorithm_unbalanced_class(s,vows_projects,places)

In [ ]:
#To do : API google cloud pour recuperer csv
#calculer score choix 1,2 ,3 pour optimiser satisfaction
# Interface streamlit/airflow
# webscrapper nom users promos beta 